# Importing Libraries

In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model, model_from_json
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image

# Setting up the model & cv2 with haar_cascade classifier

In [ ]:
model = model_from_json(open("./model.json", "r").read()) #reading the saved model from training
model.load_weights('./model.h5') #loading the weights from saved file after training
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') #initializing classifier to detect faces using haar_cascade
cap = cv2.VideoCapture(0) #initializing camera using cv2 to capture face

In [ ]:
while cap.isOpened():
    #setting up the camera frame and UI
    res, frame = cap.read()
    height, width , channel = frame.shape
    sub_img = frame[0:int(height / 9),0:int(width)]
    rect = np.ones(sub_img.shape, dtype = np.uint8) * 0
    res = cv2.addWeighted(sub_img, 0.77, rect, 0.23, 0)
    
    #styling font and label properties
    FONT = cv2.FONT_HERSHEY_TRIPLEX
    FONT_SCALE = 0
    FONT_THICKNESS = 0
    label_color = (64, 255, 0)
    label = ""
    
    #positioning
    label_dimension = cv2.getTextSize(label, FONT, FONT_SCALE, FONT_THICKNESS)[0]
    textX = int((res.shape[1] - label_dimension[0]) / 2)
    textY = int((res.shape[0] + label_dimension[1]) / 2)
    cv2.putText(res, label, (textX, textY), FONT, FONT_SCALE, (255, 255, 255), FONT_THICKNESS)
    
    #capturing faces and converting the images to grayscale
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray_image)
    
    try:
        for (x, y, w, h) in faces:
            #selecting Region of Interest(ROI) from camera
            cv2.rectangle(frame, pt1 = (x, y), pt2 = (x+w, y+h), color = (64, 255, 0),thickness =  2)
            roi_gray = gray_image[y-5 : y+h+5, x-5 : x+w+5]
            roi_gray = cv2.resize(roi_gray, (48,48))
            image_pixels = img_to_array(roi_gray) #converting image to array for prediction of emotions
            image_pixels = np.expand_dims(image_pixels, axis = 0)
            image_pixels /= 255
            predictions = model.predict(image_pixels) #using the model to predict emotions
            max_index = np.argmax(predictions[0]) #storing max prediction value to max_index
            emotions = ('Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral')
            emotion_prediction = emotions[max_index] #selecting matching prediction from emotions tuple
            
            #displaying prediction and accuracy as labels in UI
            cv2.putText(res, "Emotion: {}".format(emotion_prediction), (40, textY+5+5), FONT, 0.7, label_color, 1)
            label_violation = 'Accuracy: {}'.format(str(np.round(np.max(predictions[0]) * 100, 1)) + "%")
            cv2.putText(res, label_violation, (400, textY+5+5), FONT, 0.7, label_color, 1)
    except:
        pass
    
    frame[0:int(height / 9), 0:int(width)] = res
    cv2.imshow('Face Emotion Detector (19033540)', frame) #frame (window) title

    #exit application on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()